### Some setup

In [5]:
import tensorflow as tf
from interpretable_net import BinaryClassificationNet
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### Load data and split into labelled and unlabelled training data and validation data

In [2]:
X, y = load_breast_cancer(return_X_y=True)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train = X[:380]
y_train = y[:380]
X_val = X[380:]
y_val = y[380:]

### Initialise network

In [38]:
# define network parameters
layers = [{'dim': 20, 'activation': tf.nn.relu, 'dropout_rate': 0.5, 'batch_norm': True},
         {'dim': 20, 'activation': tf.nn.relu, 'dropout_rate': 0.5, 'batch_norm': True}]
initialiser = tf.keras.initializers.glorot_uniform()
l2_reg = 0.01
optimiser = tf.train.AdamOptimizer()
lambda_u = 0.01

# initialise network
model = BinaryClassificationNet(layers=layers,
                                 initialiser=initialiser,
                                 l2_reg=l2_reg,
                                 optimiser=optimiser,
                                 lambda_u=lambda_u)

### Train  for an initial 10 epochs

In [39]:
model.fit(X_train, 
        y_train, 
        X_val,
        y_val,
        batch_size=32, 
        num_epochs=10, 
        patience=10,
        verbose=True)

+-----------------------------------------------------------+
Running epoch 1 of 10
Training loss = 0.6034808144659588
Validation loss = 0.6291120493065112
+-----------------------------------------------------------+
Running epoch 2 of 10
Training loss = 0.5415261851231519
Validation loss = 0.5624726327165724
+-----------------------------------------------------------+
Running epoch 3 of 10
Training loss = 0.4988272597130976
Validation loss = 0.5187542240613352
+-----------------------------------------------------------+
Running epoch 4 of 10
Training loss = 0.45677981256556355
Validation loss = 0.47748963645210973
+-----------------------------------------------------------+
Running epoch 5 of 10
Training loss = 0.4165160152914101
Validation loss = 0.4374740448578325
+-----------------------------------------------------------+
Running epoch 6 of 10
Training loss = 0.3832366534067612
Validation loss = 0.40580041196059297
+-----------------------------------------------------------+

### Train for a further 10 epochs

In [40]:
# continue training
model.fit(X_train, 
        y_train, 
        X_val,
        y_val,
        batch_size=32, 
        num_epochs=10, 
        patience=10,
        verbose=True)

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
+-----------------------------------------------------------+
Running epoch 1 of 10
Training loss = 0.27415862040055033
Validation loss = 0.30106231310803977
+-----------------------------------------------------------+
Running epoch 2 of 10
Training loss = 0.2574526110412798
Validation loss = 0.2842581714263245
+-----------------------------------------------------------+
Running epoch 3 of 10
Training loss = 0.24424794217828955
Validation loss = 0.2697960946206299
+-----------------------------------------------------------+
Running epoch 4 of 10
Training loss = 0.2345743621595351
Validation loss = 0.2593972457224188
+-----------------------------------------------------------+
Running epoch 5 of 10
Training loss = 0.22534233734814302
Validation loss = 0.2503079193291284
+-----------------------------------------------------------+
Running epoch 6 of 10
Training loss = 0.21569826481588136
Validation loss = 0.239241161

### Compute training and validation loss (should be same as last epoch)

In [41]:
p_train = model.predict(X_train)
loss_train = log_loss(y_train, p_train, eps=1e-5)

p_val = model.predict(X_val)
loss_val = log_loss(y_val, p_val, eps=1e-5)

print('Training loss =', loss_train)
print('Validation loss =', loss_val)

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
Training loss = 0.1824066000441444
Validation loss = 0.20188282284603076


### Get partial derivatives of output probability wrt inputs

In [51]:
dp_dX_val = model.get_dp_dX(X_val)
dp_dX_val_mean = dp_dX_val.mean(axis=1)
feature_names = load_breast_cancer()['feature_names']
for i, feature in enumerate(feature_names):
    print('dp /', 'd(', feature, ') =', dp_dX_val_mean[i])

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
dp / d( mean radius ) = -0.02006579195149243
dp / d( mean texture ) = -0.0061539530346635726
dp / d( mean perimeter ) = -0.008221661516775688
dp / d( mean area ) = -0.023271288792602717
dp / d( mean smoothness ) = -0.013939174559588234
dp / d( mean compactness ) = -0.028622552861149113
dp / d( mean concavity ) = -0.010223967162892223
dp / d( mean concave points ) = -0.006153666534616301
dp / d( mean symmetry ) = -0.006066223220356429
dp / d( mean fractal dimension ) = -0.00920400924126928
dp / d( radius error ) = -0.004189088875621868
dp / d( texture error ) = -0.00784769964326794
dp / d( perimeter error ) = -0.005523745273239911
dp / d( area error ) = -0.0006209536872726555
dp / d( smoothness error ) = -0.01739777942420915
dp / d( compactness error ) = -0.011702424671966583
dp / d( concavity error ) = -0.027382021381830175
dp / d( concave points error ) = -0.011587683057102064
dp / d( symmetry error ) = -0.004769313265